In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import time

In [ ]:
rng = np.random.default_rng(seed=42)

# Question

Write a function called <b>simulate_fill_times</b> that takes a list of limit order quantities and a list of current bid/ask depths as input.

It should simulate fill times based on the following:

- If quantity is less than 20% of the bid/ask depth:
  - Fill time ~ Normal(0.1, 0.02) seconds
- If quantity is between 20% and 50% of depth:
  - Fill time ~ Normal(0.25, 0.05) seconds
- If quantity is over 50% of depth:
  - Fill time ~ Normal(0.5, 0.1) seconds


The function should:

- Initialize output list for fill times
- Loop through quantities and current depths:
- Calculate quantity as a percentage of current depth
- Based on percentage, sample fill time from appropriate normal distribution
- Append to output list
- Return list of simulated fill times

In [ ]:
def simulate_fill_times(quantities, depths):

  fill_times = []

  for i in range(len(quantities)):
    quantity = quantities[i]
    depth = depths[i]

    pct_of_depth = quantity / depth

    if pct_of_depth < 0.2:
      fill_time = rng.normal(0.1, 0.02)
    elif pct_of_depth < 0.5:
      fill_time = rng.normal(0.25, 0.05)
    else:
      fill_time = rng.normal(0.5, 0.1)

    fill_times.append(fill_time)

  return fill_times

In [ ]:
quantities = [100, 2000, 750, 1200]
depths = [10000, 4000, 2500, 1000]

simulate_fill_times(quantities, depths)

[0.10609434159508863,
 0.3960015893759504,
 0.2875225597903229,
 0.5940564716391215]

# Question

Write a function called <b>simulate_order_flow</b> that takes the following parameters:

- num_orders: the number of orders to generate
- max_quantity: the maximum order quantity
- tick_size: the minimum price increment

It should:

- Initialize the last price (at 100), ask price, and bid price based on a sample price and tick size (ask_price = round(last_price + tick_size, 2), bid_price = round(last_price - tick_size, 2)).

- Create an empty dictionary of lists to store the generated orders.

- Use a loop to generate "num_orders" orders:

  - Randomly select a limit or market order type (with equal probabilities)
  
  - For limit orders:
    - Randomly select bid or ask side
    - Set price at current bid or ask

  - For market orders:
    - Randomly select buy or sell side
    - Set price to 0
  
  - Randomly generate the order quantity from a list of integers betwee 1 and the "max_quantity". Use rng.integers(low, high)
    
  - Add the order details to the order list

- Return the a data frame of the order type, side, price, and quantity.

In [ ]:
def simulate_order_flow(num_orders, max_quantity, tick_size):

  last_price = 100
  ask_price = round(last_price + tick_size, 2)
  bid_price = round(last_price - tick_size, 2)

  order = {"type": [], "side": [], "price": [], "quantity": []}

  for i in range(num_orders):

    order_type = rng.choice(["limit", "market"])

    if order_type == "limit":
      side = rng.choice(["bid", "ask"])
      price = bid_price if side == "bid" else ask_price
      quantity = rng.integers(1, max_quantity)

    if order_type == "market":
      side = rng.choice(["buy", "sell"])
      price = 0
      quantity = rng.integers(1, max_quantity)

    order['type'].append(order_type)
    order['side'].append(side)
    order['price'].append(price)
    order['quantity'].append(quantity)

  return pd.DataFrame(order)

In [ ]:
num_orders = 1000
max_qty = 500
tick_size = 0.01

simulate_order_flow(num_orders, max_qty, tick_size)

,type,side,price,quantity
0,limit,bid,99.99,263
1,market,sell,0.00,380
2,market,sell,0.00,257
3,limit,ask,100.01,225
4,market,buy,0.00,92
...,...,...,...,...
995,market,sell,0.00,315
996,limit,bid,99.99,218
997,market,sell,0.00,336
998,market,sell,0.00,352


# Question
<br>
<font size="+1">
    <ul>
        <li>Consider a simplified trading problem faced by all capital market participants, large and small.</li>
        <br>
        <li>Assume you want to <b>buy</b> a stock because you believe it is a favorable investment.</li>
        <br>
        <li>At the time you decide you want to buy the stock, the limit order book has a bid price (i.e. highest price people are willing to buy) of $\$10$ and an ask price (i.e. the lowest price people are willing to sell) of $\$12$. In other words, people are willing to buy the stock at $10$ and sell the stock at $12$. The mid-price (by definition is the midpoint between the bid and the ask) is therefore $\$11$.</li>
            <br>
        <li>You must choose one of two possible choices:</li>
        <br>
        <ul>
            <li>to transact (buy) now at the ask price using a market order,</li>
            <br>
            <li>or to place a limit order at the bid price and wait some period of time, say $15$ minutes, to see if your order gets filled,</li>
            <br>
            <ul>
                <li>note that we can assume the order getting filled is a two outcome random variable (i.e. Filled or Not Filled),</li>
                <br>
                <li>if you placed a limit order and it gets filled in the period you decided to wait ($15$ minutes), then your order will get filled at the current bid price,</li>
                <br>
                <li>if you placed a limit order and it didn't get filled in the period you decided to wait ($15$ minutes), you will place a market order at the ask price $15$ minutes after you placed your limit order; this is referred to as a <i>clean-up market order</i>.</li>
                <br>
                <li>After you cancel your limit order and place the clean-up market order, assume the price you pay will be uniformly distributed with a low value of the mid price and a high value of the ask price plus $0.25$ times the time you decide to wait to place a clean-up market order (in this example it will be $0.25 \times 15$).</li>
                <br>
                <li>Because prices change (vary) over time, you must take into account the probability of your order being filled if you choose to use a limit order at the bid price, and how the price might change when you use a clean-up market order if your order doesn't get filled.</li>
                <br>
            </ul>
        </ul>
        <li>To measure the quality of your decision, you are to compute a metric called the <b>implementation shortfall</b> (i.e. <b>slippage</b>) which we will define as the difference between the price you pay when your order gets filled and the mid-price; slippage depends on the order you decide to use and it is a number we want to be small, in general.</li>
        <br>
        <li>Depending on the order type, the implementation shortfall will be</li>
        <br>
        <ul>
            <li>in the case of using a current market order, the implementation shortfall will be $P_0^{ask} - P_0^{mid}= \text{ask price} - \text{mid price}$, which is $12 - 11$ in the running example,</li>
            <br>
            <li>in the case of using a limit order at the bid price, you have two cases depending on a random simulation of if your limit order gets filled or not (i.e. a two outcome random variable):</li>
            <br>
            <ul>
                <li>if the order gets filled in the prescribed wait time ($15$ minutes) then the implementation shortfall will be $P_0^{bid} - P_0^{mid}= \text{bid price} - \text{mid price}$, which is $10 - 11$ in the running example,</li>
                    <br>
                <li>if the order does not get filled in the prescribed wait time ($15$ minutes) then the implementation shortfall will be $P_{15}^{ask} - P_0^{mid}= \text{ask price 15 mins later} - \text{mid price}$, which is assumed to be $\text{Uniform}(\text{low} = \text{mid price}, \ \text{high} = \text{ask price }+ (0.25)\text{wait time}) - 11$, where the wait time is $15$ in the running example - hint: rng.uniform(mid_price, ask_price+(0.25*wait_time)) - mid_price.</li>
                    <br>
            </ul>
        </ul>
        <li>To get an accurate estimate of shortfall, you should simulate a number of limit order scenarios and estimate the shortfall averaged over all of your simulations.</li>
        <br>
        <li>For simplicity, you can assume the quantity you want to buy is sufficiently small so that there are no effects of price impact on your optimal order decision. That is, assume the market for your security is sufficiently liquid.</li>
        <br>
        <li style="color:blue">Write a function called <b>optimal_buy_order</b> with a signature of <br><b>optimal_buy_order(bid_price, ask_price, wait_time, fill_prob_in_wait_time, num_simulations)</b>,<br> that computes the average implementation shortfall (slippage) for your two possible order placement decisions (market order and limit order) and returns the average slippage estimates in a dictionary.</li>
        <br>
        <li style="color:blue">If you reset the random seed to $42$, you should get the following results:</li>
        <br>
        <ul style="color:blue">
            <li>rng = np.random.default_rng(seed=42)</li>
            <br>
            <li>optimal_buy_order(bid_price=10, ask_price=12, wait_time=8, fill_prob_in_wait_time=0.2, num_simulations=10_000) <br> $\longrightarrow$ {'initial_market_order_slippage': 1.0, 'average_limit_order_slippage': 0.9884036474082528}<br></li>
            <br>
            <li>optimal_buy_order(bid_price=10, ask_price=12, wait_time=4, fill_prob_in_wait_time=0.2, num_simulations=10_000) <br> $\longrightarrow$ {'initial_market_order_slippage': 1.0, 'average_limit_order_slippage': 0.6058792816131945}<br></li>
            <br>
            <li>optimal_buy_order(bid_price=10, ask_price=12, wait_time=15, fill_prob_in_wait_time=0.2, num_simulations=10_000) <br> $\longrightarrow$ {'initial_market_order_slippage': 1.0, 'average_limit_order_slippage': 1.7006321138565643}</li>
            <br>
        </ul>
        <li>Notice how it can be better to use limit orders over market orders if you have smaller wait time.</li>
        <br>
    </ul>
</font>

$\square$

In [ ]:
def optimal_buy_order(bid_price, ask_price, wait_time, fill_prob_in_wait_time, num_simulations):
    # Define the midprice
    mid_price = (bid_price + ask_price) / 2
    # Create a dictionary to compute average slippage in limit-bid or market-ask scenarios
    order_slippage_dict = {}
    # We know the initial market order slippage is ask - mid
    order_slippage_dict['initial_market_order_slippage'] = ask_price - mid_price

    # Simulate limit orders to estimate average slippage
    limit_order_slippage = []
    for sim in range(num_simulations):
        # Randomly generate if your order is filled or not
        limit_order_status = rng.choice(['Filled', 'Not Filled'],
                                        p=[fill_prob_in_wait_time, 1 - fill_prob_in_wait_time])
        # Event that your order gets filled
        if limit_order_status == 'Filled':
            # then your slippage will be the bid - mid
            limit_order_slippage.append(bid_price - mid_price)
        # Event that your order doesn't get filled in your prescribed wait time
        else:
            # then your slippage will be Uniform[mid, ask+(1/4)wait_time] - mid
            limit_order_slippage.append(rng.uniform(mid_price, ask_price+(0.25*wait_time)) - mid_price)

    # Compute average limit order slippage and save it in the dictionary
    order_slippage_dict['average_limit_order_slippage'] = sum(limit_order_slippage) / len(limit_order_slippage)

    return order_slippage_dict

In [ ]:
rng = np.random.default_rng(seed=42)

In [ ]:
optimal_buy_order(bid_price=10, ask_price=12, wait_time=8, fill_prob_in_wait_time=0.2, num_simulations=10_000)

{'initial_market_order_slippage': 1.0,
 'average_limit_order_slippage': 0.9884036474082528}

In [ ]:
optimal_buy_order(bid_price=10, ask_price=12, wait_time=4, fill_prob_in_wait_time=0.2, num_simulations=10_000)

{'initial_market_order_slippage': 1.0,
 'average_limit_order_slippage': 0.6058792816131945}

In [ ]:
optimal_buy_order(bid_price=10, ask_price=12, wait_time=15, fill_prob_in_wait_time=0.2, num_simulations=10_000)

{'initial_market_order_slippage': 1.0,
 'average_limit_order_slippage': 1.7006321138565643}

In [ ]:
# is there anything that might be useful to keep track of during this estimation, such as the fill price or wait time, etc.?

# Question
<br>
<font size="+1">
    <ul>
        <li>For this question, assume you have a limit order book that has been initialized.</li>
        <br>
        <li><b>Order flow</b> refers to new orders being placed in the limit order book, regardless of the order being a limit or market order; so order flow leads to an altered state of the limit order book.</li>
        <br>
        <li>Assume there are only four possible orders that can enter the market:</li>
        <br>
        <ul>
            <li>"Market-Bid", which refers to a market order placed at the bid price (this order would be placed by someone who wants to sell stock now at the bid price),</li>
            <br>
            <li>"Market-Ask", which refers to a market order placed at the ask price (this order would be placed by someone who wants to buy stock now at the ask price),</li>
            <br>
            <li>"Limit-Bid", which refers to a limit order placed at the bid price (this order would be placed by someone who wants to buy at a more favorable price and is willing to wait to achieve that favorable bid price),</li>
            <br>
            <li>"Limit-Ask", which refers to a limit order placed at the ask price (this order would be placed by someone who wants to sell at a more favorable price and is willing to wait to achieve that favorable ask price).</li>
            <br>
            <li>These four order types can be decomposed into the form: <i style="color:blue">order_type + "-" + side_of_market</i>, where order type refers to a limit or a market order and the side of the market refers to the bid price or the ask price.</li>
            <br>
        </ul>
        <li>Incoming orders (i.e. order flow) can be influenced by the current state of the limit order book.</li>
        <br>
        <ul>
            <li>In particular, assume that the particular order being placed is <a href="https://finance.zacks.com/can-tell-direction-stock-price-looking-bid-vs-ask-volume-2758.html">influenced by the <b>total liquidity</b> at the bid price and the ask price</a>, where <b>total liquidity</b> refers to the sum of all the order quantities in the limit order book at the bid price or at the ask price, respectively.</li>
            <br>
            <li>Specifically, assume the following three scenarios that determine the order flow (i.e. incoming order) - this is going to walk you through how the function should be structured:</li>
            <br>
            <ul>
                <li>if the total liquidity at the bid price is less than the total liquidity at the ask price (this would signal more people want to sell and it is likely that the price will go down), then the order type will be either a market or limit order with a probability of a market order being the (total liquidity at the ask / (sum of the total liquidity at the bid and the ask)) - hint: use rng.choice();</li>
                <br>
                <ul>
                    <li>in this particular scenario ($\text{total_liquidity_at_bid} < \text{total_liquidity_at_ask}$), if there is a market order, then the specific order type should be a market-bid since people will sell, otherwise it should be a limit-ask since more people will line up to sell,</li>
                    <br>
                    <li>once you know the order type all you need is the order quantity, which you can assume is a random integer between $1$ and $15$ - hint: use rng.integers(1,15),</li>
                    <br>
            </ul>
            <li>if the total liquidity at the bid price is greater than the total liquidity at the ask price (this would signal more people want to buy and it is likely that the price will go up), then the order type will be either a market or limit order with a probability of a market order being the (total liquidity at the bid / (sum of the total liquidity at the bid and the ask)) - hint: use rng.choice();</li>
            <br>
            <ul>
                <li>in this particular scenario ($\text{total_liquidity_at_bid} > \text{total_liquidity_at_ask}$), if there is a market order, then the specific order type should be a market-ask since people will buy, otherwise it should be a limit-bid since more people will line up to buy,</li>
                <br>
                <li>once you know the order type all you need is the order quantity, which you can assume is a random integer between $1$ and $15$ - hint: use rng.integers(1,15),</li>
                <br>
            </ul>
            <li>in the final case where the total liquidity at the bid price equals the total liquidity at the ask price (this would not have any signal about if more people want to buy or sell), then the order type will be either a market or limit order with a probability of a market order being $0.5$ (equal chance of being limit or market order) - hint: use rng.choice();</li>
                <br>
                <ul>
                    <li>in this particular scenario ($\text{total_liquidity_at_bid} == \text{total_liquidity_at_ask}$), the specific order type can be anything, in which case you should also randomly choose (with equal probabilities) if it is at the bid or ask price,</li>
                    <br>
                    <li>once you know the order type all you need is the order quantity, which you can assume is a random integer between $1$ and $15$ in all scenarios - hint: use rng.integers(1,15),</li>
                    <br>
            </ul>
        </ul>
        <li>After all this logic is written, you should return something of the form $$(\text{order_type}+\text{'-'}+\text{side_of_market}, \  \text{rng.integers}(1,15)).$$</li>
        <br>
        <li style="color:blue">You are to write a function called <b>order_flow</b> with a signature of <b>order_flow(limit_order_book)</b> that returns a two element tuple where the first element is the specific type of order, which can be one of four possbilities: Limit-Bid, Limit-Ask, Market-Bid, Market-Ask, and the second element is quantity to be ordered, which will be assumed to be a randomly generated integer in the range of 1 to 15.</li>
        <br>
        <li style="color:blue">If you reset the random seed to $42$ and run the following code, you should get the following results:</li>
        <br>
        <ul style="color:blue">
            <li>rng = np.random.default_rng(seed=42)</li>
            <br>
            <li>initial_LOB = initialize_limit_order_book(bid_price=10, bid_queue=rng.integers(1, 30, size=7), ask_price=12, ask_queue=rng.integers(1,30, size=10)),<br></li>
            <br>
            <li>order_flow(limit_order_book=initial_LOB) <br> $\longrightarrow$ ('Market-Bid', 2).<br></li>
            <br>
        </ul>
        <li>Hint: you can think about structuring your code in the following format if you are stuck (this is not the only way to do it):</li>
            <br>
            <ul>
                <li>### Stuff</li>
                <br>
                <li>if total_liquidity_at_bid < total_liquidity_at_ask: <br> ### Do Stuff</li>
                <br>
                <ul>
                    <li>if order_type == 'Market': <br> ### Do stuff</li>
                    <br>
                    <li>else: <br> ### Do stuff</li>
                    <br>
                </ul>
                <li>elif total_liquidity_at_bid > total_liquidity_at_ask: <br> ### Do Stuff</li>
                <br>
                <ul>
                    <li>if order_type == 'Market': <br> ### Do stuff</li>
                    <br>
                    <li>else: <br> ### Do stuff</li>
                    <br>
                </ul>
                <li>else: <br> ### Do Stuff</li>
                <br>
            </ul>
    </ul>
</font>

$\square$

In [ ]:
# Simulate new data (artificial orders) in order to gauge how the existing book
# would process the newly simulated orders.
# This includes random types, random prices, random times through the day, and random quantities.

In [ ]:
# Single order flow generation in a simplified setting

def order_flow(limit_order_book):
    bid_price = list(limit_order_book['buys'].keys())[0]
    ask_price = list(limit_order_book['sells'].keys())[0]

    total_liquidity_at_bid = limit_order_book['buys'][bid_price].sum()
    total_liquidity_at_ask = limit_order_book['sells'][ask_price].sum()


    if total_liquidity_at_bid < total_liquidity_at_ask:
        # this means price is likely to go down and we can expect a market order at bid with a higher probability than a limit order to sell
        prob_of_market_order = total_liquidity_at_ask / (total_liquidity_at_ask + total_liquidity_at_bid)
        order_type = rng.choice(['Market', 'Limit'], p=[prob_of_market_order, 1 - prob_of_market_order])

        if order_type == 'Market':
            # for simplicity, assume we will have a randomly generated quantity
            orders_to_be_processed = (order_type+'-Bid', rng.integers(1,15))
        else:
            orders_to_be_processed = (order_type+'-Ask', rng.integers(1,15))

    elif total_liquidity_at_bid > total_liquidity_at_ask:
        # this means price is likely to go up and we can expect a market order at ask with a higher probability than a limit order to buy
        prob_of_market_order = total_liquidity_at_bid / (total_liquidity_at_ask + total_liquidity_at_bid)
        order_type = rng.choice(['Market', 'Limit'], p=[prob_of_market_order, 1 - prob_of_market_order])

        if order_type == 'Market':
            # for simplicity, assume we will have a randomly generated quantity
            orders_to_be_processed = (order_type+'-Ask', rng.integers(1,15))
        else:
            orders_to_be_processed = (order_type+'-Bid', rng.integers(1,15))

    else:
        # this means no new information has come out and we can expect a market order on either side or a limit order on either side with equal probabilities
        order_type = rng.choice(['Market', 'Limit'], p=[0.5, 0.5])
        side_of_market = rng.choice(['Bid', 'Ask'], p=[0.5, 0.5])

        orders_to_be_processed = (order_type+'-'+side_of_market, rng.integers(1,15))

    return orders_to_be_processed

# Question

Write a function called <b>smart_execution</b> that breaks a large order into smaller chunks in a way that minimizes slippage based on simulated market conditions.

The function accepts:

- total_shares: the total order size
- start_price: the starting stock price

It should:

- Simulate market conditions each execution by randomly generating:
  - a current bid/ask spread (use rng.integers(1, 10))
  - a temporary price impact measure (use rng.uniform(0.01, 0.03))
  
- Sequentially execute smaller orders, dynamically adjusting size based on:
  
  - current bid/ask spread
     - assume it is randomly generated as rng.integers(1, 10)
  - temporary price impact measure
    - assume it is calculated as order size times an impact measure, which is randomly generated from a uniform distribution between [0.01, 0.03]
  - shares remaining
  - In particular, for the order size, use
    - order_size = int(max(100, shares_left/spread))

- Track average price and total slippage

- Return the percentage slippage from start_price

In [ ]:
def smart_execution(total_shares, start_price):

  shares_left = total_shares
  total_cost = 0

  while shares_left > 0:

    # Simulate market conditions
    spread = rng.integers(1, 10)
    impact_measure = rng.uniform(0.01, 0.03)

    # Determine next order size
    order_size = int(max(100, shares_left/spread))

    # Calculate price impact
    impact = order_size * impact_measure

    # Update shares and cost
    shares_left -= order_size
    total_cost += (start_price + impact) * order_size

  avg_price = total_cost / total_shares
  slippage = ((avg_price - start_price) / start_price) * 100 # to convert from decimal to percentage

  return slippage

In [ ]:
total_shares = 1000000
start_price = 205

smart_execution(total_shares, start_price)

1516.4217923277602